In [1]:
## Preprocessing

In [2]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
! pip install tensorflow



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


In [4]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
#  YOUR CODE GOES HERE

application_df = application_df.drop(['EIN', 'NAME'], axis=1)

In [5]:
# Determine the number of unique values in each column.
#  YOUR CODE GOES HERE
uniqueValues = application_df.nunique()

print(uniqueValues)

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64


In [6]:
# Look at APPLICATION_TYPE value counts for binning
#  YOUR CODE GOES HERE
applicationType = application_df['APPLICATION_TYPE'].value_counts()

print(applicationType)


T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64


In [7]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
#  YOUR CODE GOES HERE
cutoff_value = 500 

# Create a list of application types to be replaced
application_types_to_replace = applicationType[applicationType < cutoff_value].index.tolist()

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [8]:
# Look at CLASSIFICATION value counts for binning
#  YOUR CODE GOES HERE
classification_counts = application_df['CLASSIFICATION'].value_counts()

print(classification_counts)

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64


In [9]:
# You may find it helpful to look at CLASSIFICATION value counts >1
#  YOUR CODE GOES HERE
classification_gt_1 = classification_counts[classification_counts > 1]

print(classification_gt_1)

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64


In [10]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
#  YOUR CODE GOES HERE
classifications_to_replace = classification_counts[classification_counts < cutoff_value].index.tolist()

application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].apply(lambda x: 'Other' if x in classifications_to_replace else x)


# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
Other     1484
C7000      777
Name: CLASSIFICATION, dtype: int64

In [11]:
# Convert categorical data to numeric with `pd.get_dummies`
#  YOUR CODE GOES HERE
# Select the columns with categorical data
categorical_columns = ['APPLICATION_TYPE', 'AFFILIATION', 'CLASSIFICATION', 'USE_CASE', 'ORGANIZATION', 'STATUS', 'INCOME_AMT', 'SPECIAL_CONSIDERATIONS']

# Use pd.get_dummies to one-hot encode the categorical columns
df_encoded = pd.get_dummies(application_df[categorical_columns])

# Concatenate the one-hot encoded columns with the original DataFrame
application_df_encoded = pd.concat([application_df, df_encoded], axis=1)

# Drop the original categorical columns
application_df_encoded.drop(categorical_columns, axis=1, inplace=True)

# Display the resulting DataFrame
print(application_df_encoded.head())



   ASK_AMT  IS_SUCCESSFUL  APPLICATION_TYPE_Other  APPLICATION_TYPE_T10  \
0     5000              1                       0                     1   
1   108590              1                       0                     0   
2     5000              0                       0                     0   
3     6692              1                       0                     0   
4   142590              1                       0                     0   

   APPLICATION_TYPE_T19  APPLICATION_TYPE_T3  APPLICATION_TYPE_T4  \
0                     0                    0                    0   
1                     0                    1                    0   
2                     0                    0                    0   
3                     0                    1                    0   
4                     0                    1                    0   

   APPLICATION_TYPE_T5  APPLICATION_TYPE_T6  APPLICATION_TYPE_T7  ...  \
0                    0                    0                  

In [12]:
# Split our preprocessed data into our features and target arrays

X = application_df.drop('IS_SUCCESSFUL', axis=1)  # Features (all columns except the target)
y = application_df['IS_SUCCESSFUL'] 

# Split the preprocessed data into a training and testing dataset
#  YOUR CODE GOES HERE
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [13]:
# Use get_dummies to convert categorical columns to one-hot encoded columns
X_train_encoded = pd.get_dummies(X_train)

# Use the same column names for one-hot encoding in the test set
X_test_encoded = pd.get_dummies(X_test)
X_test_encoded = X_test_encoded.reindex(columns=X_train_encoded.columns, fill_value=0)




In [14]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train_encoded)

# Scale the data
X_train_scaled = X_scaler.transform(X_train_encoded)
X_test_scaled = X_scaler.transform(X_test_encoded)

## Compile, Train and Evaluate the Model

In [15]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
# YOUR CODE GOES HERE

nn = tf.keras.models.Sequential()

# First hidden layer
# YOUR CODE GOES HERE
nn.add(tf.keras.layers.Dense(units=80, input_dim=44, activation='relu'))

# Second hidden layer (if necessary)
# YOUR CODE GOES HERE
nn.add(tf.keras.layers.Dense(units=30, activation='relu'))

# Output layer
# YOUR CODE GOES HERE
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 80)                3600      
                                                                 
 dense_1 (Dense)             (None, 30)                2430      
                                                                 
 dense_2 (Dense)             (None, 1)                 31        
                                                                 
Total params: 6061 (23.68 KB)
Trainable params: 6061 (23.68 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [16]:
# Compile the model
#  YOUR CODE GOES HERE
# Compile the model
nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])



In [17]:
# Train the model
#  YOUR CODE GOES HERE

model_history = nn.fit(X_train_scaled, y_train, epochs=100, verbose=2, validation_split=0.2)


Epoch 1/100
686/686 - 1s - loss: 0.5716 - accuracy: 0.7179 - val_loss: 0.5568 - val_accuracy: 0.7369 - 752ms/epoch - 1ms/step
Epoch 2/100
686/686 - 0s - loss: 0.5561 - accuracy: 0.7277 - val_loss: 0.5520 - val_accuracy: 0.7336 - 449ms/epoch - 655us/step
Epoch 3/100
686/686 - 0s - loss: 0.5533 - accuracy: 0.7280 - val_loss: 0.5465 - val_accuracy: 0.7389 - 417ms/epoch - 608us/step
Epoch 4/100
686/686 - 0s - loss: 0.5518 - accuracy: 0.7288 - val_loss: 0.5454 - val_accuracy: 0.7382 - 331ms/epoch - 483us/step
Epoch 5/100
686/686 - 0s - loss: 0.5509 - accuracy: 0.7294 - val_loss: 0.5449 - val_accuracy: 0.7371 - 370ms/epoch - 539us/step
Epoch 6/100
686/686 - 0s - loss: 0.5488 - accuracy: 0.7319 - val_loss: 0.5463 - val_accuracy: 0.7387 - 369ms/epoch - 538us/step
Epoch 7/100
686/686 - 0s - loss: 0.5485 - accuracy: 0.7307 - val_loss: 0.5462 - val_accuracy: 0.7398 - 310ms/epoch - 451us/step
Epoch 8/100
686/686 - 0s - loss: 0.5479 - accuracy: 0.7315 - val_loss: 0.5456 - val_accuracy: 0.7378 - 326

Epoch 65/100
686/686 - 0s - loss: 0.5371 - accuracy: 0.7376 - val_loss: 0.5462 - val_accuracy: 0.7380 - 307ms/epoch - 447us/step
Epoch 66/100
686/686 - 0s - loss: 0.5365 - accuracy: 0.7373 - val_loss: 0.5484 - val_accuracy: 0.7394 - 303ms/epoch - 442us/step
Epoch 67/100
686/686 - 0s - loss: 0.5367 - accuracy: 0.7371 - val_loss: 0.5481 - val_accuracy: 0.7400 - 308ms/epoch - 448us/step
Epoch 68/100
686/686 - 0s - loss: 0.5366 - accuracy: 0.7383 - val_loss: 0.5501 - val_accuracy: 0.7380 - 307ms/epoch - 447us/step
Epoch 69/100
686/686 - 0s - loss: 0.5359 - accuracy: 0.7374 - val_loss: 0.5476 - val_accuracy: 0.7392 - 302ms/epoch - 441us/step
Epoch 70/100
686/686 - 0s - loss: 0.5364 - accuracy: 0.7372 - val_loss: 0.5509 - val_accuracy: 0.7389 - 321ms/epoch - 468us/step
Epoch 71/100
686/686 - 0s - loss: 0.5361 - accuracy: 0.7378 - val_loss: 0.5490 - val_accuracy: 0.7369 - 350ms/epoch - 510us/step
Epoch 72/100
686/686 - 0s - loss: 0.5362 - accuracy: 0.7384 - val_loss: 0.5485 - val_accuracy: 0.

In [18]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

215/215 - 0s - loss: 0.5657 - accuracy: 0.7249 - 71ms/epoch - 332us/step
Loss: 0.565682053565979, Accuracy: 0.7249271273612976


In [19]:
# Export our model to HDF5 file
#  YOUR CODE GOES HERE

# Save the model to an HDF5 file
nn.save("AlphabetSoupCharity.keras")


In [20]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

# Define the model
nn_optimized = Sequential()

# Add the first hidden layer with more neurons
nn_optimized.add(Dense(units=128, activation='relu', input_dim=X_train_scaled.shape[1]))

# Add a dropout layer to prevent overfitting
nn_optimized.add(Dropout(0.2))

# Add a second hidden layer with more neurons and a different activation function
nn_optimized.add(Dense(units=64, activation='tanh'))

# Add the output layer
nn_optimized.add(Dense(units=1, activation='sigmoid'))

# Compile the model
nn_optimized.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Display the model structure
nn_optimized.summary()

# Train the optimized model
optimized_model_history = nn_optimized.fit(X_train_scaled, y_train, epochs=100, verbose=2, validation_split=0.2)


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 128)               5760      
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_4 (Dense)             (None, 64)                8256      
                                                                 
 dense_5 (Dense)             (None, 1)                 65        
                                                                 
Total params: 14081 (55.00 KB)
Trainable params: 14081 (55.00 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/100
686/686 - 1s - loss: 0.5755 - accuracy: 0.7133 - val_loss: 0.5550 - val_accuracy: 0.7334 - 613ms/epoch - 893us/step
Epoch 2/100
686/686 - 

Epoch 57/100
686/686 - 0s - loss: 0.5396 - accuracy: 0.7365 - val_loss: 0.5491 - val_accuracy: 0.7336 - 343ms/epoch - 499us/step
Epoch 58/100
686/686 - 0s - loss: 0.5403 - accuracy: 0.7352 - val_loss: 0.5413 - val_accuracy: 0.7400 - 340ms/epoch - 495us/step
Epoch 59/100
686/686 - 0s - loss: 0.5399 - accuracy: 0.7361 - val_loss: 0.5433 - val_accuracy: 0.7394 - 338ms/epoch - 493us/step
Epoch 60/100
686/686 - 0s - loss: 0.5404 - accuracy: 0.7364 - val_loss: 0.5430 - val_accuracy: 0.7403 - 346ms/epoch - 504us/step
Epoch 61/100
686/686 - 0s - loss: 0.5399 - accuracy: 0.7350 - val_loss: 0.5413 - val_accuracy: 0.7383 - 404ms/epoch - 589us/step
Epoch 62/100
686/686 - 0s - loss: 0.5398 - accuracy: 0.7351 - val_loss: 0.5423 - val_accuracy: 0.7385 - 491ms/epoch - 716us/step
Epoch 63/100
686/686 - 0s - loss: 0.5394 - accuracy: 0.7349 - val_loss: 0.5439 - val_accuracy: 0.7396 - 380ms/epoch - 554us/step
Epoch 64/100
686/686 - 0s - loss: 0.5400 - accuracy: 0.7355 - val_loss: 0.5408 - val_accuracy: 0.

In [21]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

215/215 - 0s - loss: 0.5657 - accuracy: 0.7249 - 71ms/epoch - 331us/step
Loss: 0.565682053565979, Accuracy: 0.7249271273612976
